# iBioFoundry - Ark Plate Creation - OT2
author: Camillo Moschner | version: 2.1 | date:06.10.2022 | license = (undefined; standard copyright laws apply)

# 1- Metadata

In [1]:
metadata = {
    'protocolName': '# Automated Liquid Handling for Molarity-adjusted Parts/Ark Plate Creation using the OT2',
    'author': 'Camillo Moschner <cm967@cam.ac.uk> / <camillo.moschner@gmail.com>',
    'description': 'Programme for the automated creation of an ark_plate',
    'apiLevel': '2.12',
    'date': '23.05.2023',
    'pipette_configuration':{'left':'p20_single_gen2',
                             'right':'p300_single_gen2'}}

## Molaritization Master Inputs
The following cell requires changing between experimental setups (and, for simplicity for the user, is supposed to be the only part of the protocol that requires any modification).

In [2]:
# establish whether you are performing a simulation (e.g. at the design stage) or are executing the script on the OT-2
script_mode = 'simulation' # 'simulation' or 'execution'
# input data
experiment_name = 'PREEx'
parts_prep_csv = 'input0_GoldenGates_parts_prep.csv'
# reagent: water
water_rack_format = 'opentrons_6_tuberack_falcon_50ml_conical'
water1_position = 'A1'
water1_volume = 15.0 # units: ml
# sample: miniprepped DNA
miniprep_rack_format = 'opentrons_24_tuberack_eppendorf_1.5ml_safelock_snapcap' # 'biorad_96_wellplate_200ul_pcr' #'opentrons_24_tuberack_eppendorf_1.5ml_safelock_snapcap' 
miniprep_dead_vol = 15 # units: µl (i.e. the amount of liquid the OT-2 cannot reach)
ark_plate_format = 'biorad_96_wellplate_200ul_pcr' # 'corning_384_wellplate_112ul_flat' | 'biorad_384_wellplate_50ul' | biorad_96_wellplate_200ul_pcr #NOT STANDARD-DEFINED: '384_PCR_plate Framestar - 4titude'
ark_plate_fill_start = 'A1'
ark_plate_column_usage = 'all' # allows you to selectively only use 'odd'- or 'even'-numbered columns or 'all'; useful for 0.2 ml tubes w/ snapcaps

# 2- Import Statements

In [3]:
import numpy as np
import pandas as pd
from IPython.display import display, clear_output, Audio
from copy import deepcopy
from datetime import date
import time
import os

from iBioFoundry_helper import *
from opentrons import protocol_api, execute, simulate

In [4]:
# establish sole Jupyter Notebook control 
if script_mode == 'execution':
    try:
        os.system("systemctl stop opentrons-robot-server")
        protocol = execute.get_protocol_api(metadata['apiLevel']);
    except:
        protocol = execute.get_protocol_api(metadata['apiLevel']);
    print("You are operating this script in exectution mode!\nYou should see the lights turn on and hear the gantry homing!")
elif script_mode == 'simulation':
    protocol = simulate.get_protocol_api(metadata['apiLevel']);
    print("You are operating this script in simulation mode!")
protocol.set_rail_lights(True)
protocol.home()

/Users/camillomoschner/.opentrons/robot_settings.json not found. Loading defaults
/Users/camillomoschner/.opentrons/deck_calibration.json not found. Loading defaults


You are operating this script in simulation mode!


# 3- Function Definitions

In [5]:
# smart careful transfer
def smart_transfer_liquid(transfer_vol, aspiration_pos,dispensation_pos,
                          pick_up_tip=True, touch_tip_before=True,
                          cyclical_mix_before=None,mix_before_reps=2,mix_rate=0.5,
                          asp_bot_clearance=1.5,dispens_bot_clearance=1.5,
                          asp_rate=0.8,disp_rate=0.5,
                          cyclical_mix_after=None,mix_after=None,
                          dispens_top_offset=-2, blow_out=False, air_gap=False, touch_tip=True, touch_miniscus=None,
                          drop_tip=True):
    """Liquid transfer that automatically identifies which tip to use.
    """
    if (transfer_vol >= p20_left.min_volume) & (transfer_vol <= p20_left.max_volume):
        pipette_to_use = p20_left
    elif (transfer_vol > p300_right.min_volume) & (transfer_vol <= p300_right.max_volume):
        pipette_to_use = p300_right
    else:
        print(f"WARNING: transfer volume not manageable")
    if pick_up_tip==True:
        pipette_to_use.pick_up_tip()
    if cyclical_mix_before != None: # give tuple (repetitions, mix_volume, well_bottom_clearance)
        for i in np.arange(cyclical_mix_before[0]):
            pipette_to_use.aspirate(cyclical_mix_before[1],aspiration_pos.bottom(cyclical_mix_before[2]))
            pipette_to_use.dispense(cyclical_mix_before[1],aspiration_pos.bottom(cyclical_mix_before[3]))
    if mix_before_reps != 0:
        pipette_to_use.mix(mix_before_reps,transfer_vol*0.8,aspiration_pos.bottom(asp_bot_clearance),mix_rate)
    # aspiration
    pipette_to_use.aspirate(transfer_vol,aspiration_pos.bottom(asp_bot_clearance), asp_rate)
    protocol.max_speeds['Z'] = 10  # limit z axis to 50 mm/s to avoid solution adhereing to outer tip surface
    pipette_to_use.move_to(aspiration_pos.top())
    protocol.max_speeds['Z'] = None # reset z axis speed
    if air_gap != False:
        pipette_to_use.air_gap(air_gap)
    protocol.delay(seconds=1)
    if touch_tip_before == True:
        pipette_to_use.touch_tip(v_offset=-2, speed=150)
    # dispensation
    pipette_to_use.dispense(transfer_vol,dispensation_pos.bottom(dispens_bot_clearance),disp_rate)
    if cyclical_mix_after != None: # give tuple (repetitions, mix_volume, well_bottom_clearance)
        for i in np.arange(cyclical_mix_after[0]):
            pipette_to_use.aspirate(cyclical_mix_after[1],dispensation_pos.bottom(cyclical_mix_after[2]))
            pipette_to_use.dispense(cyclical_mix_after[1],dispensation_pos.bottom(cyclical_mix_after[3]))
    if mix_after != None:
        pipette_to_use.mix(mix_after[0],mix_after[1], dispensation_pos.bottom(mix_after[2]),mix_rate)
    pipette_to_use.move_to(dispensation_pos.top())
    protocol.delay(seconds=2)
    if blow_out == True:
        pipette_to_use.blow_out(dispensation_pos.top(dispens_top_offset))
    if touch_tip == True:
        pipette_to_use.touch_tip(v_offset=dispens_top_offset, speed=150)
    if touch_miniscus != None:
        protocol.max_speeds['Z'] = 10  # limit z axis to 50 mm/s to avoid solution adhereing to outer tip surface
        pipette_to_use.move_to(dispensation_pos.bottom(touch_miniscus))
        protocol.max_speeds['Z'] = None # reset z axis speed
    if drop_tip==True:
        pipette_to_use.drop_tip()

---
# 4- Deck assignments

In [6]:
"""Deck creation"""
deck_slot_list =list(np.arange(1,12))
deck_slot_list.append('bin')
deck_slot_df = pd.DataFrame( np.flip(np.array(deck_slot_list).reshape(4,3), axis=0) )

## i. Labware
Resource: [Opentrons Labware Library Webpage](https://labware.opentrons.com/?_gl=1*1iyc1t4*_ga*MTU1MTM1NzU5MS4xNjE2OTczMzY5*_ga_GNSMNLW4RY*MTYzMDYxNzY1NS4yMC4wLjE2MzA2MTc2NTUuMA..&_ga=2.110416899.767557364.1630617656-1551357591.1616973369)
### a. Modules

In [7]:
temp_mod_name  = 'temperature module gen2'
temp_mod = protocol.load_module(temp_mod_name,'3')
deck_slot_df.iloc[3,2] = temp_mod_name

### b. Racks & Plates

In [8]:
tiprack20a = protocol.load_labware('opentrons_96_tiprack_20ul','8')
tiprack20b = protocol.load_labware('opentrons_96_tiprack_20ul','11')
tiprack300a = protocol.load_labware('opentrons_96_tiprack_300ul','10')
# racks & plates
miniprep_rack = protocol.load_labware(miniprep_rack_format,'1') # assigned in code cell no. 5
water_rack = protocol.load_labware(water_rack_format,'4') 
ark_plate = temp_mod.load_labware(ark_plate_format,'5') # assigned in code cell no. 5

In [9]:
labware_list = [tiprack20a,tiprack20b, tiprack300a, 
                miniprep_rack, ark_plate, water_rack]
_ =[update_labware(deck_slot_df, labware_item) for labware_item in labware_list]
# visually inspect deck
print("\nFinal OT-2 Deck Configuration:");(deck_slot_df)


Final OT-2 Deck Configuration:


,0,1,2
0,10-opentrons_96_tiprack_300ul,11-opentrons_96_tiprack_20ul,bin
1,7,8-opentrons_96_tiprack_20ul,9
2,4-opentrons_6_tuberack_falcon_50ml_conical,5,6
3,1-opentrons_24_tuberack_eppendorf_1.5ml_safelo...,2,temperature module gen2


## ii. Pipettes

Resource: [Opentrons Pipettes API Webpage](https://docs.opentrons.com/v2/new_pipette.html)

In [10]:
p20_left = protocol.load_instrument('p20_single_gen2','left', tip_racks=[tiprack20a,tiprack20b])
p300_right = protocol.load_instrument('p300_single_gen2','right', tip_racks=[tiprack300a])

## iii. Reagent assignment

### In-Notebook Calibration

In [11]:
pipette_to_calibrate = p20_left # p20_left | p300_right
labware_to_calibrate_to = miniprep_rack # miniprep_rack | water_rack | ark_plate
labware_position_to_test = 'A1'

In [12]:
# step 0: pick up tip for the pipette you want to calibrate and move to the labware to be calibrated
pipette_to_calibrate.pick_up_tip()
pipette_to_calibrate.move_to(labware_to_calibrate_to[labware_position_to_test].top())

<InstrumentContext: p20_single_v2.0 in LEFT>

In [13]:
# step 1: set_offset based on identifying in which direction the tip needs to move
# (can be an iderative process as this cell immediately moves to the new position)
labware_to_calibrate_to.set_offset(x=(0.),y=(0.),z=(0.))
pipette_to_calibrate.move_to(labware_to_calibrate_to[labware_position_to_test].top())

<InstrumentContext: p20_single_v2.0 in LEFT>

In [14]:
pipette_to_calibrate.move_to(labware_to_calibrate_to['C1'].top())

<InstrumentContext: p20_single_v2.0 in LEFT>

In [15]:
# step 2: return tip (if you haven'crashed it
pipette_to_calibrate.return_tip()

<InstrumentContext: p20_single_v2.0 in LEFT>

---
# 5- LH Step Calculations 
## Miniprep File (input0) Reading

In [16]:
parts_prep_info_df = pd.read_csv(parts_prep_csv)
sample_no = len(parts_prep_info_df)
# calculate nM of existing DNA samples
average_MW_of_bp = 617.96 # source: https://nebiocalculator.neb.com/#!/formulas
nanomolar_c = [ round( ((parts_prep_info_df.iloc[row,:]['concentration (ng/ul)']*1_000_000) / (parts_prep_info_df.iloc[row,:]['length']*average_MW_of_bp+36.04)),3) 
               for row in range(sample_no) ]
parts_prep_info_df.insert(3,'concentration (nM)',nanomolar_c)
parts_prep_info_df

,name,length,concentration (ng/ul),concentration (nM),available sample to dilute (ul),desired_concentration (nM)
0,P_T7(BBF10K_003378)_15nM,2412,124.1,83.258,20,15.0
1,P_T7_lacO(BBF10K_003379)_15nM,2427,121.1,80.743,20,15.0
2,RBS_BT1_BCD2(BBF10K_003384),2413,138.6,92.947,20,15.0
3,RBS_BT1_BCD2_DsbA(BBF10K_003391)_15nM,2849,225.4,128.024,20,15.0
4,RBS_BT1_BCD2_OmpT(BBF10K_003392)_15nM,2441,112.8,74.777,20,15.0
5,RBS_BT1_BCD2_pelB(BBF10K_003393)_15nM,2426,147.1,98.119,20,15.0
6,Ntag1_R5(BBF10K_003399)_30nM,2420,154.6,103.377,20,30.0
7,Ntag2_TEVcut(BBF10K_003418)_30nM,2410,153.3,102.933,20,30.0
8,Ntag_His_TEVcut(BBF10K_003407_15nM,2427,137.2,91.477,20,15.0
9,CDS_Eco31I(BBF10K_003292)_15nM,3866,237.2,99.286,20,15.0


## Dilution Calculations

In [17]:
# adapt to sample throughput choice
miniprep_max_well_vol, miniprep_plate_format = identify_plate(miniprep_rack_format, 'max_well_vol'), identify_plate(miniprep_rack_format, 'plate_format')
ark_plate_max_well_vol, ark_plate_format = identify_plate(ark_plate_format, 'max_well_vol'), identify_plate(ark_plate_format, 'plate_format')

sample_utilized = []
dilution_water_per_sample = []
nanodrop_checkup = []
# calculate dilutioner volume per ul of sample
dilution_vol_per_ul = [ round( (parts_prep_info_df.iloc[row,:]['concentration (nM)'] / (parts_prep_info_df.iloc[row,:]['desired_concentration (nM)']))-1,3) for row in range(sample_no) ]
# calculate individual sample utilisation requirements
for idx in range(sample_no):
    sample_data = parts_prep_info_df.iloc[idx,:]
    available_sample = sample_data['available sample to dilute (ul)']
    # scenario 1 - volume too large for format chosen, i.e. 96-well plate or 1.5 ml Eppi
    while dilution_vol_per_ul[idx]*available_sample+available_sample > ark_plate_max_well_vol:
        available_sample-=1
    # scenario 2 - dilution volume not pipettable, i.e. <1 ul / too small to be pipettable or negative, i.e. starting concentration too low
    if dilution_vol_per_ul[idx]*available_sample < 2: # less than two ul are not accurate enough to be pipetted -> TEST OUT BORDER
        print(f"\nWARNING: sample \'{sample_data['name']}\' is not concentrated enough!\n")
        available_sample = 0
    sample_utilized.append(available_sample)
    dilution_water_per_sample.append(dilution_vol_per_ul[idx] * available_sample)
    nanodrop_checkup.append(round((sample_data['concentration (ng/ul)'] * available_sample) / (dilution_vol_per_ul[idx]*available_sample + available_sample), 2))
# update preparation information with water needed and sample utilized
parts_prep_info_df['dilution_water (ul)'], parts_prep_info_df['sample utilized (ul)'] = dilution_water_per_sample, sample_utilized
parts_prep_info_df['dilution_water (ul)'] = round(parts_prep_info_df['dilution_water (ul)'],2)
parts_prep_info_df['total volume (ul)'] = round(parts_prep_info_df['dilution_water (ul)'] + parts_prep_info_df['sample utilized (ul)'],2)
parts_prep_info_df['nanodrop checkup (ng/ul)'] = nanodrop_checkup
# parts_prep_info_df

In [18]:
"""Liquid handling position and index assignmnent
"""
# create plate tracking objects
ark_plate_obj = Plate(ark_plate_format,'ark_plate',only_columns='all')
try: # in case you want to add new parts to a plate already used
    sample_definitions_layout_dir = f'0_molarity_adjusted_parts_info_{experiment_name}{os.path.sep}1_molarity_adjusted_parts_info.csv'
    sample_definitions_df = pd.read_csv(sample_definitions_layout_dir)
    sample_definitions_df['position_tuple'] = sample_definitions_df['moladj_parts_positions'].apply(lambda name: (name[0],int(name[1:])))
    for part_info in [sample_definitions_df.iloc[row,:] for row in range(len(sample_definitions_df))]:
        ark_plate_obj.layout.loc[part_info['position_tuple']]=part_info['name']
    print(f"The software found an already existing part_plate.\n")
except:
    pass
miniprep_plate_obj = Plate(miniprep_plate_format,'miniprep_plate')

# fill plate oject layouts with parts
miniprep_plate_obj.layout = extend_fill_plate_df_with_list(miniprep_plate_obj.layout, 
                                                           parts_prep_info_df.name.to_list(), fill_start_position='A1', fill_first='columns')
ark_plate_obj.layout = extend_fill_plate_df_with_list(ark_plate_obj.layout, parts_prep_info_df.name.to_list(), fill_start_position=ark_plate_fill_start, fill_first='columns')
ark_plate_obj.layout

,1,2,3,4,5,6,7,8,9,10,11,12
A,P_T7(BBF10K_003378)_15nM,Ntag_His_TEVcut(BBF10K_003407_15nM,DV_pTi_7.5nM,0,0,0,0,0,0,0,0,0
B,P_T7_lacO(BBF10K_003379)_15nM,CDS_Eco31I(BBF10K_003292)_15nM,DV_pTi_5nM,0,0,0,0,0,0,0,0,0
C,RBS_BT1_BCD2(BBF10K_003384),CDS_EcoRI(BBF10K_003281)_15nM,0,0,0,0,0,0,0,0,0,0
D,RBS_BT1_BCD2_DsbA(BBF10K_003391)_15nM,CDS_NotI(BBF10K_003300)_15nM,0,0,0,0,0,0,0,0,0,0
E,RBS_BT1_BCD2_OmpT(BBF10K_003392)_15nM,CDS_PstI(BBF10K_003283)_15nM,0,0,0,0,0,0,0,0,0,0
F,RBS_BT1_BCD2_pelB(BBF10K_003393)_15nM,CDS_SapI(BBF10K_003295)_15nM,0,0,0,0,0,0,0,0,0,0
G,Ntag1_R5(BBF10K_003399)_30nM,CDS_XbaI(BBF10K_003298)_15nM,0,0,0,0,0,0,0,0,0,0
H,Ntag2_TEVcut(BBF10K_003418)_30nM,T_TZ(BBF10K_003477)_15nM,0,0,0,0,0,0,0,0,0,0


In [19]:
# find Opentrons locations for all parts
miniprep_positions = parts_prep_info_df['name'].apply(lambda sample_name: miniprep_rack[find_df_coordinates(miniprep_plate_obj.layout,sample_name)] )
moladj_parts_positions = parts_prep_info_df['name'].apply(lambda sample_name: ark_plate[find_df_coordinates(ark_plate_obj.layout,sample_name)] )
"""
Liquid handling position and index assignmnent
"""
# create plate tracking objects
ark_plate_obj = Plate(ark_plate_format,'ark_plate',only_columns=ark_plate_column_usage)
try: # in case you want to add new parts to a plate already used before
    sample_definitions_layout_dir = f'0_molarity_adjusted_parts_info_{experiment_name}{os.path.sep}1_molarity_adjusted_parts_info.csv'
    sample_definitions_df = pd.read_csv(sample_definitions_layout_dir)
    sample_definitions_df['position_tuple'] = sample_definitions_df['moladj_parts_positions'].apply(lambda name: (name[0],int(name[1:])))
    for part_info in [sample_definitions_df.iloc[row,:] for row in range(len(sample_definitions_df))]:
        ark_plate_obj.layout.loc[part_info['position_tuple']]=part_info['name']
    print(f"The software found an already existing part_plate.\n")
except:
    pass
miniprep_plate_obj = Plate(miniprep_plate_format,'miniprep_plate')
# fill plate oject layouts with parts
miniprep_plate_obj.layout = extend_fill_plate_df_with_list(miniprep_plate_obj.layout, parts_prep_info_df.name.to_list(), fill_start_position='A1', fill_first='columns')
ark_plate_obj.layout = extend_fill_plate_df_with_list(ark_plate_obj.layout, parts_prep_info_df.name.to_list(), fill_start_position=ark_plate_fill_start, fill_first='columns')
# find Opentrons locations for all parts
miniprep_positions = parts_prep_info_df['name'].apply(lambda sample_name: miniprep_rack[find_df_coordinates(miniprep_plate_obj.layout,sample_name)] )
moladj_parts_positions = parts_prep_info_df['name'].apply(lambda sample_name: ark_plate[find_df_coordinates(ark_plate_obj.layout,sample_name)] )
# update sample info df with positions
parts_prep_info_df['miniprep_positions'] = miniprep_positions
parts_prep_info_df['moladj_parts_positions'] = moladj_parts_positions
parts_prep_info_df['prep_date'] = [str(date.today()) for i in range(sample_no)]

parts_prep_info_df#.style.background_gradient(subset=['sample utilized (ul)'], cmap='viridis')

,name,length,concentration (ng/ul),concentration (nM),available sample to dilute (ul),desired_concentration (nM),dilution_water (ul),sample utilized (ul),total volume (ul),nanodrop checkup (ng/ul),miniprep_positions,moladj_parts_positions,prep_date
0,P_T7(BBF10K_003378)_15nM,2412,124.1,83.258,20,15.0,91.02,20,111.02,22.36,A1 of Opentrons 24 Tube Rack with Eppendorf 1....,A1 of 5 on Temperature Module GEN2 on 3,2023-06-04
1,P_T7_lacO(BBF10K_003379)_15nM,2427,121.1,80.743,20,15.0,87.66,20,107.66,22.50,B1 of Opentrons 24 Tube Rack with Eppendorf 1....,B1 of 5 on Temperature Module GEN2 on 3,2023-06-04
2,RBS_BT1_BCD2(BBF10K_003384),2413,138.6,92.947,20,15.0,103.92,20,123.92,22.37,C1 of Opentrons 24 Tube Rack with Eppendorf 1....,C1 of 5 on Temperature Module GEN2 on 3,2023-06-04
3,RBS_BT1_BCD2_DsbA(BBF10K_003391)_15nM,2849,225.4,128.024,20,15.0,150.70,20,170.70,26.41,D1 of Opentrons 24 Tube Rack with Eppendorf 1....,D1 of 5 on Temperature Module GEN2 on 3,2023-06-04
4,RBS_BT1_BCD2_OmpT(BBF10K_003392)_15nM,2441,112.8,74.777,20,15.0,79.70,20,99.70,22.63,A2 of Opentrons 24 Tube Rack with Eppendorf 1....,E1 of 5 on Temperature Module GEN2 on 3,2023-06-04
5,RBS_BT1_BCD2_pelB(BBF10K_003393)_15nM,2426,147.1,98.119,20,15.0,110.82,20,130.82,22.49,B2 of Opentrons 24 Tube Rack with Eppendorf 1....,F1 of 5 on Temperature Module GEN2 on 3,2023-06-04
6,Ntag1_R5(BBF10K_003399)_30nM,2420,154.6,103.377,20,30.0,48.92,20,68.92,44.86,C2 of Opentrons 24 Tube Rack with Eppendorf 1....,G1 of 5 on Temperature Module GEN2 on 3,2023-06-04
7,Ntag2_TEVcut(BBF10K_003418)_30nM,2410,153.3,102.933,20,30.0,48.62,20,68.62,44.68,D2 of Opentrons 24 Tube Rack with Eppendorf 1....,H1 of 5 on Temperature Module GEN2 on 3,2023-06-04
8,Ntag_His_TEVcut(BBF10K_003407_15nM,2427,137.2,91.477,20,15.0,101.96,20,121.96,22.50,A3 of Opentrons 24 Tube Rack with Eppendorf 1....,A2 of 5 on Temperature Module GEN2 on 3,2023-06-04
9,CDS_Eco31I(BBF10K_003292)_15nM,3866,237.2,99.286,20,15.0,112.38,20,132.38,35.84,B3 of Opentrons 24 Tube Rack with Eppendorf 1....,B2 of 5 on Temperature Module GEN2 on 3,2023-06-04


In [20]:
water_transfer_info_p20 = parts_prep_info_df.loc[parts_prep_info_df['dilution_water (ul)'] <= p20_left.max_volume]
water_transfer_info_p300 = parts_prep_info_df.loc[parts_prep_info_df['dilution_water (ul)'] > p20_left.max_volume]
summary_sheet_df = parts_prep_info_df[['name', 'desired_concentration (nM)', 'total volume (ul)']].copy()
summary_sheet_df.rename(columns={'desired_concentration (nM)': 'concentration (nM)'},inplace=True)
summary_sheet_df['moladj_parts_positions'] = parts_prep_info_df['moladj_parts_positions'].apply(lambda well: well.well_name) # for Nanodrop checkup
summary_sheet_df['total volume (ul)'] = parts_prep_info_df['total volume (ul)'].apply(lambda volume: volume-1) # for Nanodrop checkup
summary_sheet_df['ark_plate_format'] =[ark_plate_format for i in range(sample_no)]
summary_sheet_df['prep_date'] = [str(date.today()) for i in range(sample_no)]
try: # in case you want to add new parts to a plate already used before
    summary_sheet_df = pd.concat([sample_definitions_df,summary_sheet_df])
    summary_sheet_df.reset_index(drop=True,inplace=True)
except:
    pass

In [21]:
summary_sheet_df

,name,concentration (nM),total volume (ul),moladj_parts_positions,ark_plate_format,prep_date
0,P_T7(BBF10K_003378)_15nM,15.0,110.02,A1,96,2023-06-04
1,P_T7_lacO(BBF10K_003379)_15nM,15.0,106.66,B1,96,2023-06-04
2,RBS_BT1_BCD2(BBF10K_003384),15.0,122.92,C1,96,2023-06-04
3,RBS_BT1_BCD2_DsbA(BBF10K_003391)_15nM,15.0,169.70,D1,96,2023-06-04
4,RBS_BT1_BCD2_OmpT(BBF10K_003392)_15nM,15.0,98.70,E1,96,2023-06-04
5,RBS_BT1_BCD2_pelB(BBF10K_003393)_15nM,15.0,129.82,F1,96,2023-06-04
6,Ntag1_R5(BBF10K_003399)_30nM,30.0,67.92,G1,96,2023-06-04
7,Ntag2_TEVcut(BBF10K_003418)_30nM,30.0,67.62,H1,96,2023-06-04
8,Ntag_His_TEVcut(BBF10K_003407_15nM,15.0,120.96,A2,96,2023-06-04
9,CDS_Eco31I(BBF10K_003292)_15nM,15.0,131.38,B2,96,2023-06-04


## Visual Plate Inspection

In [22]:
print(f"\n__Miniprep Rack__")
display(miniprep_plate_obj.layout)
print(f"\n__Ark Plate Rack__")
display(ark_plate_obj.layout)


__Miniprep Rack__


,1,2,3,4,5,6
A,P_T7(BBF10K_003378)_15nM,RBS_BT1_BCD2_OmpT(BBF10K_003392)_15nM,Ntag_His_TEVcut(BBF10K_003407_15nM,CDS_PstI(BBF10K_003283)_15nM,DV_pTi_7.5nM,0
B,P_T7_lacO(BBF10K_003379)_15nM,RBS_BT1_BCD2_pelB(BBF10K_003393)_15nM,CDS_Eco31I(BBF10K_003292)_15nM,CDS_SapI(BBF10K_003295)_15nM,DV_pTi_5nM,0
C,RBS_BT1_BCD2(BBF10K_003384),Ntag1_R5(BBF10K_003399)_30nM,CDS_EcoRI(BBF10K_003281)_15nM,CDS_XbaI(BBF10K_003298)_15nM,0,0
D,RBS_BT1_BCD2_DsbA(BBF10K_003391)_15nM,Ntag2_TEVcut(BBF10K_003418)_30nM,CDS_NotI(BBF10K_003300)_15nM,T_TZ(BBF10K_003477)_15nM,0,0



__Ark Plate Rack__


,1,2,3,4,5,6,7,8,9,10,11,12
A,P_T7(BBF10K_003378)_15nM,Ntag_His_TEVcut(BBF10K_003407_15nM,DV_pTi_7.5nM,0,0,0,0,0,0,0,0,0
B,P_T7_lacO(BBF10K_003379)_15nM,CDS_Eco31I(BBF10K_003292)_15nM,DV_pTi_5nM,0,0,0,0,0,0,0,0,0
C,RBS_BT1_BCD2(BBF10K_003384),CDS_EcoRI(BBF10K_003281)_15nM,0,0,0,0,0,0,0,0,0,0
D,RBS_BT1_BCD2_DsbA(BBF10K_003391)_15nM,CDS_NotI(BBF10K_003300)_15nM,0,0,0,0,0,0,0,0,0,0
E,RBS_BT1_BCD2_OmpT(BBF10K_003392)_15nM,CDS_PstI(BBF10K_003283)_15nM,0,0,0,0,0,0,0,0,0,0
F,RBS_BT1_BCD2_pelB(BBF10K_003393)_15nM,CDS_SapI(BBF10K_003295)_15nM,0,0,0,0,0,0,0,0,0,0
G,Ntag1_R5(BBF10K_003399)_30nM,CDS_XbaI(BBF10K_003298)_15nM,0,0,0,0,0,0,0,0,0,0
H,Ntag2_TEVcut(BBF10K_003418)_30nM,T_TZ(BBF10K_003477)_15nM,0,0,0,0,0,0,0,0,0,0


## Summary Spreadsheets

In [23]:
current_date = str(date.today())
current_date_mini = ''.join(current_date[2:].split('-'))
run_name = '0_ark_plate_info_'+experiment_name
save_spreadsheet_answer = input("Do you want to save the summary spreadsheets? (yes / no)\n answer:")
if save_spreadsheet_answer == 'yes':
    try:
        os.makedirs(run_name)
    except:
        pass
    # save prepped equimolar sample information
    parts_prep_info_df.to_csv(f"{run_name}{os.path.sep}0_full_prep_info.csv",index=False)
    summary_sheet_df.to_csv(f"{run_name}{os.path.sep}1_ark_plate_info.csv",index=False)
    print(f"The summary spreadsheets have been saved to a new folder called \'{run_name}{os.path.sep}\' !")
else:
    print(f"\nYou have not saved the summary spreadsheets!\n")

Do you want to save the summary spreadsheets? (yes / no)
 answer: yes


The summary spreadsheets have been saved to a new folder called '0_ark_plate_info_PREEx/' !


---
# 6- Execution
## i. Water Dispensation

In [24]:
%%time
# p20 water dispensations
water1 = water_rack[water1_position]
aspiration_clearance=5
print(f"Water transfer...  into {ark_plate.name} in position {ark_plate.parent}")
if len(water_transfer_info_p20) != 0:
    x= [print(f" - {round(water,1)} µl -> {position.well_name}") for water, position in zip(water_transfer_info_p20['dilution_water (ul)'].to_list(), water_transfer_info_p20['moladj_parts_positions'].to_list())]
    p20_left.transfer(
        water_transfer_info_p20['dilution_water (ul)'].to_list(),
        water1.bottom(aspiration_clearance),
        water_transfer_info_p20['moladj_parts_positions'].to_list())
    print(f"   ...using {p20_left.model}\n")
# p300 water dispensations
if len(water_transfer_info_p300) != 0:
    x= [print(f" - {round(water,1)} µl -> {position.well_name}") for water, position in zip(water_transfer_info_p300['dilution_water (ul)'].to_list(), water_transfer_info_p300['moladj_parts_positions'].to_list())]
    p300_right.transfer(
        water_transfer_info_p300['dilution_water (ul)'].to_list(),
        water1.bottom(aspiration_clearance),
        water_transfer_info_p300['moladj_parts_positions'].to_list())
    print(f"   ...using {p300_right.model}\n")

Water transfer...  into 5 in position Temperature Module GEN2 on 3
 - 91.0 µl -> A1
 - 87.7 µl -> B1
 - 103.9 µl -> C1
 - 150.7 µl -> D1
 - 79.7 µl -> E1
 - 110.8 µl -> F1
 - 48.9 µl -> G1
 - 48.6 µl -> H1
 - 102.0 µl -> A2
 - 112.4 µl -> B2
 - 124.8 µl -> C2
 - 96.5 µl -> D2
 - 122.8 µl -> E2
 - 86.7 µl -> F2
 - 81.2 µl -> G2
 - 140.0 µl -> H2
 - 64.5 µl -> A3
 - 104.2 µl -> B3
   ...using p300_single_v2.0

CPU times: user 10.7 ms, sys: 886 µs, total: 11.6 ms
Wall time: 11.2 ms


## ii. Miniprep Dispensation

In [25]:
%%time
# use positions to loop through DataFrame
print(f"Miniprepped samples transferred:")
for idx, miniprep_well_location in enumerate(parts_prep_info_df['miniprep_positions']):
    # identify destination_location and miniprep_volume to be taken for the current sample
    destination_location = parts_prep_info_df.loc[parts_prep_info_df['miniprep_positions']==miniprep_well_location] ['moladj_parts_positions'].iloc[0]
    miniprep_volume_utilized = parts_prep_info_df.loc[parts_prep_info_df['miniprep_positions']==miniprep_well_location] ['sample utilized (ul)'].iloc[0]
    # update and execute transfer
    print(f" - {miniprep_volume_utilized} µl {parts_prep_info_df['name'].iloc[idx]} ({miniprep_well_location.well_name}) -> {destination_location.well_name}.")
    smart_transfer_liquid(miniprep_volume_utilized, miniprep_well_location,destination_location, 
                          asp_rate=0.5,asp_bot_clearance=1.2, dispens_bot_clearance=2,
                          mix_before_reps=0, cyclical_mix_before=(1,10, 1.3,3),
                          blow_out=True, touch_tip_before=False, touch_tip=False)

Miniprepped samples transferred:
 - 20 µl P_T7(BBF10K_003378)_15nM (A1) -> A1.
 - 20 µl P_T7_lacO(BBF10K_003379)_15nM (B1) -> B1.
 - 20 µl RBS_BT1_BCD2(BBF10K_003384) (C1) -> C1.
 - 20 µl RBS_BT1_BCD2_DsbA(BBF10K_003391)_15nM (D1) -> D1.
 - 20 µl RBS_BT1_BCD2_OmpT(BBF10K_003392)_15nM (A2) -> E1.
 - 20 µl RBS_BT1_BCD2_pelB(BBF10K_003393)_15nM (B2) -> F1.
 - 20 µl Ntag1_R5(BBF10K_003399)_30nM (C2) -> G1.
 - 20 µl Ntag2_TEVcut(BBF10K_003418)_30nM (D2) -> H1.
 - 20 µl Ntag_His_TEVcut(BBF10K_003407_15nM (A3) -> A2.
 - 20 µl CDS_Eco31I(BBF10K_003292)_15nM (B3) -> B2.
 - 20 µl CDS_EcoRI(BBF10K_003281)_15nM (C3) -> C2.
 - 20 µl CDS_NotI(BBF10K_003300)_15nM (D3) -> D2.
 - 20 µl CDS_PstI(BBF10K_003283)_15nM (A4) -> E2.
 - 20 µl CDS_SapI(BBF10K_003295)_15nM (B4) -> F2.
 - 20 µl CDS_XbaI(BBF10K_003298)_15nM (C4) -> G2.
 - 20 µl T_TZ(BBF10K_003477)_15nM (D4) -> H2.
 - 15 µl DV_pTi_7.5nM (A5) -> A3.
 - 15 µl DV_pTi_5nM (B5) -> B3.
CPU times: user 46.3 ms, sys: 4.7 ms, total: 51 ms
Wall time: 49.9 ms

# Shutdown

In [26]:
try:
    p20_left.drop_tip()
except:
    pass
try:
    p300_right.drop_tip()
except:
    pass
protocol.home()

In [27]:
# complete tip usage check:
_=[display(show_rack_usage(labware)[1]) for labware in labware_list if 'Opentrons 96 Tip Rack' in str(labware)]

Opentrons 96 Tip Rack 20 µL on 8:
 -> 19 tips used:


,1,2,3,4,5,6,7,8,9,10,11,12
A,x,x,x,0,0,0,0,0,0,0,0,0
B,x,x,x,0,0,0,0,0,0,0,0,0
C,x,x,x,0,0,0,0,0,0,0,0,0
D,x,x,0,0,0,0,0,0,0,0,0,0
E,x,x,0,0,0,0,0,0,0,0,0,0
F,x,x,0,0,0,0,0,0,0,0,0,0
G,x,x,0,0,0,0,0,0,0,0,0,0
H,x,x,0,0,0,0,0,0,0,0,0,0


Opentrons 96 Tip Rack 20 µL on 11:
 -> 0 tips used:


,1,2,3,4,5,6,7,8,9,10,11,12
A,0,0,0,0,0,0,0,0,0,0,0,0
B,0,0,0,0,0,0,0,0,0,0,0,0
C,0,0,0,0,0,0,0,0,0,0,0,0
D,0,0,0,0,0,0,0,0,0,0,0,0
E,0,0,0,0,0,0,0,0,0,0,0,0
F,0,0,0,0,0,0,0,0,0,0,0,0
G,0,0,0,0,0,0,0,0,0,0,0,0
H,0,0,0,0,0,0,0,0,0,0,0,0


Opentrons 96 Tip Rack 300 µL on 10:
 -> 1 tips used:


,1,2,3,4,5,6,7,8,9,10,11,12
A,x,0,0,0,0,0,0,0,0,0,0,0
B,0,0,0,0,0,0,0,0,0,0,0,0
C,0,0,0,0,0,0,0,0,0,0,0,0
D,0,0,0,0,0,0,0,0,0,0,0,0
E,0,0,0,0,0,0,0,0,0,0,0,0
F,0,0,0,0,0,0,0,0,0,0,0,0
G,0,0,0,0,0,0,0,0,0,0,0,0
H,0,0,0,0,0,0,0,0,0,0,0,0
